## Visualise stokes annulus model (flow etc)

In [16]:
# to fix trame issue
import nest_asyncio
nest_asyncio.apply()
import os

In [17]:
import petsc4py
import underworld3 as uw
import numpy as np
import sympy

In [44]:
ls -trl output/cartesian/ | tail

total 0
drwxr-xr-x  1585 lmoresi  staff  50720 Aug 20 07:40 Ra1e7.0/
drwxr-xr-x@ 1208 lmoresi  staff  38656 Aug 20 10:19 Ra1e5.0/
drwxr-xr-x@ 1815 lmoresi  staff  58080 Aug 20 18:03 Ra1e6.0/
drwxr-xr-x@  808 lmoresi  staff  25856 Aug 20 20:12 Ra1e4.0/
drwxr-xr-x@  808 lmoresi  staff  25856 Aug 20 21:04 Ra1e3.0/
drwxr-xr-x@  404 lmoresi  staff  12928 Aug 22 11:54 Ra1e2.0/


In [45]:
ls -trl output/cartesian/Ra1e2.0/ | tail

-rw-r--r--@  1 lmoresi  staff  234920 Aug 22 11:54 cartesian_Ra1e2.0_res10.mesh.T.00097.h5
-rw-r--r--@  1 lmoresi  staff    3076 Aug 22 11:54 cartesian_Ra1e2.0_res10.mesh.00097.xdmf
-rw-r--r--@  1 lmoresi  staff  137272 Aug 22 11:54 cartesian_Ra1e2.0_res10.mesh.U.00098.h5
-rw-r--r--@  1 lmoresi  staff   51880 Aug 22 11:54 cartesian_Ra1e2.0_res10.mesh.P.00098.h5
-rw-r--r--@  1 lmoresi  staff  234920 Aug 22 11:54 cartesian_Ra1e2.0_res10.mesh.T.00098.h5
-rw-r--r--@  1 lmoresi  staff    3076 Aug 22 11:54 cartesian_Ra1e2.0_res10.mesh.00098.xdmf
-rw-r--r--@  1 lmoresi  staff  137272 Aug 22 11:54 cartesian_Ra1e2.0_res10.mesh.U.00099.h5
-rw-r--r--@  1 lmoresi  staff   51880 Aug 22 11:54 cartesian_Ra1e2.0_res10.mesh.P.00099.h5
-rw-r--r--@  1 lmoresi  staff  234920 Aug 22 11:54 cartesian_Ra1e2.0_res10.mesh.T.00099.h5
-rw-r--r--@  1 lmoresi  staff    3076 Aug 22 11:54 cartesian_Ra1e2.0_res10.mesh.00099.xdmf


In [46]:
# This is a bit repetitive, sorry 

checkpoint_dir = f"output/cartesian/Ra1e2.0"
name = "cartesian_Ra1e2.0"
resolution = 10

checkpoint_base = f"{name}_res{resolution}"
meshfile = os.path.join(checkpoint_dir, checkpoint_base) + ".mesh.00000.h5"
image_dir = os.path.join(checkpoint_dir,"images")
os.makedirs(image_dir, exist_ok=True)

In [47]:
mesh = uw.discretisation.Mesh(meshfile, 
                              coordinate_system_type=uw.coordinates.CoordinateSystemType.CARTESIAN)
    
x = mesh.N.x
y = mesh.N.y

v_soln = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
t_soln = uw.discretisation.MeshVariable("T", mesh, 1, degree=3)
flux = uw.discretisation.MeshVariable(r"dTdz", mesh, 1, degree=1)

In [48]:
mesh.dm.view()

DM Object: uw_output/cartesian/Ra1e2.0/cartesian_Ra1e2.0_res10.mesh.00000.h5 1 MPI process
  type: plex
uw_output/cartesian/Ra1e2.0/cartesian_Ra1e2.0_res10.mesh.00000.h5 in 2 dimensions:
  Number of 0-cells per rank: 793
  Number of 1-cells per rank: 2236
  Number of 2-cells per rank: 1444
Labels:
  depth: 3 strata with value/size (0 (793), 1 (2236), 2 (1444))
  All_Boundaries: 1 strata with value/size (1001 (140))
  Bottom: 1 strata with value/size (11 (119))
  Elements: 1 strata with value/size (99999 (2097))
  Left: 1 strata with value/size (14 (19))
  Right: 1 strata with value/size (13 (19))
  Top: 1 strata with value/size (12 (119))
  UW_Boundaries: 5 strata with value/size (1001 (140), 11 (119), 12 (119), 13 (19), 14 (19))
  celltype: 3 strata with value/size (0 (793), 1 (2236), 3 (1444))
Field U:
  adjacency FEM
Field T:
  adjacency FEM
Field dTdz:
  adjacency FEM


In [49]:
steps = range(0,30,1)

In [50]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pl = pv.Plotter(window_size=[1000, 500])

for step in steps:

    try:
        v_soln.read_timestep(checkpoint_base, "U", step, outputPath=checkpoint_dir)
        t_soln.read_timestep(checkpoint_base, "T", step, outputPath=checkpoint_dir)
        print(f"Plotting step {step}")

    except:
        print(f"Failed to read step {step} correctly")
        continue

    pl.clear()

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym[0])
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln, alpha=None)
    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln, alpha=None)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym[0])
    
    # point sources at cell centres
    skip = 10
    points = np.zeros((mesh._centroids[::skip].shape[0], 3))
    points[:, 0] = mesh._centroids[::skip, 0]
    points[:, 1] = mesh._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)

    pvstream = pvmesh.streamlines_from_source(
        point_cloud,
        vectors="V",
        integration_direction="both",
        max_time=1,
        surface_streamlines=True,
    )
    
    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=True,
        scalars="T",
        use_transparency=False,
        opacity=1.0,
        # clim=[0.0,1.0],
        # show_scalar_bar=False
    )



    pl.add_mesh(pvmesh, cmap="RdBu_r", scalars="T", opacity=0.5, show_edges=True, edge_color="Black",
        edge_opacity=0.5, show_scalar_bar=True)    

    pl.add_mesh(pvstream)

    pl.camera_position = [(3.0, 0.5, 6.750994846396983),
                          (3.0, 0.5, 0.0),
                          (0.0, 1.0, 0.0)]

    imagefile = os.path.join(image_dir, checkpoint_base) + f"_V_T_{step}.png"
    
    pl.screenshot(filename=imagefile,
                  window_size=(1000, 500),
                  return_img=False)



Plotting step 0
Plotting step 1
Plotting step 2
Plotting step 3
Plotting step 4
Plotting step 5
Plotting step 6
Plotting step 7
Plotting step 8
Plotting step 9
Plotting step 10
Plotting step 11
Plotting step 12
Plotting step 13
Plotting step 14
Plotting step 15
Plotting step 16
Plotting step 17
Plotting step 18
Plotting step 19
Plotting step 20
Plotting step 21
Plotting step 22
Plotting step 23
Plotting step 24
Plotting step 25
Plotting step 26
Plotting step 27
Plotting step 28
Plotting step 29


In [43]:
pl.show(jupyter_backend="trame")


Widget(value='<iframe src="http://localhost:60457/index.html?ui=P_0x319431a10_5&reconnect=auto" class="pyvista…

In [11]:
## Calculate heat flux, evaluate at surface — proxy for boundary layer thickness

In [12]:
t_soln.stats()

(5611,
 302.0304989713002,
 -244.7466025206416,
 1683.3386469668976,
 1694693.1297279652,
 42735.52602685222,
 570.5174332558662)

In [13]:
0/0

ZeroDivisionError: division by zero

In [ ]:
flux_solver = uw.systems.Projection(mesh, flux)

# Conductive flux only !
flux = -mesh.vector.gradient(t_soln.sym[0]).dot(mesh.CoordinateSystem.unit_e_0)
# vertical_flux *= sympy.exp(-100*(r-1)**2) 

flux_solver.uw_function = flux
flux_solver.smoothing = 1.0e-3
flux_solver.solve()

In [ ]:
flux

In [ ]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["dTdz"] = vis.scalar_fn_to_pv_points(pvmesh, flux.sym[0])
pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
pvmesh.point_data["V"] -= pvmesh.point_data["V"].mean()

velocity_points = vis.meshVariable_to_pv_cloud(v_soln)
velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, v_soln.sym)

# point sources at cell centres
skip = 8
points = np.zeros((mesh._centroids[::skip].shape[0], 3))
points[:, 0] = mesh._centroids[::skip, 0]
points[:, 1] = mesh._centroids[::skip, 1]
point_cloud = pv.PolyData(points)

pvstream = pvmesh.streamlines_from_source(
    point_cloud,
    vectors="V",
    integration_direction="both",
    max_time=0.5,
)

pl = pv.Plotter(window_size=[1000, 1000])

pl.add_mesh(
    pvmesh,
    cmap="Reds",
    edge_color="Black",
    show_edges=True,
    scalars="dTdz",
    use_transparency=False,
    opacity=1.0,
)

# 
# pl.add_mesh(pvmesh, "Black", "wireframe",  opacity=0.1)

# pl.add_points(point_cloud, color="White", point_size=3.0, opacity=0.25)

# pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=0.0001, show_scalar_bar=True)
pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)
    
# pl.remove_scalar_bar("V")

imagefile = os.path.join(image_dir, checkpoint_base) + f"_dTdr_{step}.png"

pl.screenshot(filename=imagefile, window_size=(750, 750), return_img=False)
# OR


In [ ]:
pl.show()